<a href="https://colab.research.google.com/github/nujudaly/T5/blob/main/Nujud_Task2_Text_Classification_ar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Classification Exam

Welcome to the Text Classification Practical Exam. In this exam, you will be tasked with building, training, and evaluating an NLP model to classify text data. You are provided with a labeled dataset containing both the text and its corresponding class labels.

Your objective is to develop a model that accurately predicts the class of the given text. Make sure to follow best practices in data preprocessing, model selection, and evaluation to achieve optimal results.

Good luck!
___

# Install and Import Needed Libraries

You can use `pyarabic` or any other library to pre-process and clean the Arabic text.

In [ ]:
pip install pyarabic

In [ ]:
import nltk

# Download the Dataset

Please note that you are allowed to take a subset of this dataset, the reason for that is it might take a long time to train the model on.

In [ ]:
!kaggle datasets download -d khaledzsa/sanad
!unzip sanad.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/sanad
License(s): unknown
 84% 60.0M/71.4M [00:00<00:00, 164MB/s]
100% 71.4M/71.4M [00:00<00:00, 148MB/s]
Archive:  sanad.zip
  inflating: sanad.csv               


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/sanad.csv")

# Data Exploration

Before diving into preprocessing and model building, it’s important to first explore the dataset to understand its structure, distribution, and key characteristics. This step will help you gain insights into the data and guide your decisions in subsequent steps. Here’s what to consider:

1. **Inspect the Data**:
   Start by looking at the first few rows of the dataset to get a sense of its structure. Check the columns, data types, and a few sample entries. This helps to ensure that the data is loaded correctly and gives you an initial overview of the content.

2. **Check for Missing Values**:
   Identify if there are any missing values in the dataset.

3. **Distribution of Labels**:
   Examine the distribution of the target labels (classes).

4. **Text Data Characteristics (Bonus)**:
   Analyze the length of the text data. It is useful to calculate the number of words or characters in each text sample to understand how long the texts are. This will help you set a suitable `max_length` for tokenization and padding later. You can plot a histogram of text lengths to visualize the distribution.

5. **Common Words and Vocabulary (Bonus)**:
   Explore the most frequent words in the text data.

In [ ]:
df.head(10)

,text,label
0,https://example.com/resource/الشاٌرقة -ْ محمِد...,Culture
1,https://example.com/resource/اَنِطٌلقّتَ ٍفٍيّ...,Culture
2,https://example.com/resource/أُقيًمٌتِ مِساءُ ...,Culture
3,https://example.com/resource/بٍاسُمةَ يًوٌنٍس ...,Culture
4,https://example.com/resource/قُرر اَتحِاد اًلْ...,Culture
5,https://example.com/resource/تتّنٌاّول َاِلِرو...,Culture
6,https://example.com/resource/أعِلُنَتُ ُهٍيئة ...,Culture
7,https://example.com/resource/توفٌي َأمس ِاَلُأ...,Culture
8,https://example.com/resource/نظم ْمَرٍكزْ َاِل...,Culture
9,https://example.com/resource/يختتم فيً تٌمامٌ ...,Culture


In [ ]:
sample = df['text'].head(1)
sample

,text
0,https://example.com/resource/الشاٌرقة -ْ محمِد...


AttributeError: 'Series' object has no attribute 'max_length'

In [ ]:
df.isna().sum()

,0
text,0
label,0


In [ ]:
df['label']

,label
0,Culture
1,Culture
2,Culture
3,Culture
4,Culture
...,...
39880,Tech
39881,Tech
39882,Tech
39883,Tech


In [ ]:
df['label'].values

array(['Culture', 'Culture', 'Culture', ..., 'Tech', 'Tech', 'Tech'],
      dtype=object)

In [ ]:
df['label'].unique()

array(['Culture', 'Finance', 'Medical', 'Politics', 'Religion', 'Sports',
       'Tech'], dtype=object)

# Data Cleaning and Preprocessing

In this section, we will focus on cleaning and filtering the dataset, preparing it for the text classification task. We will implement the following steps:

1. **Remove missing values**:
   First, we eliminate any rows with missing values to ensure the dataset is complete and consistent.

2. **Filter by text length (Bonus)**:
   To maintain a uniform dataset, we will filter the text samples by a specified word count range. This ensures that the texts are neither too short to lack context nor too long to introduce unnecessary complexity.

3. **Arabic stopwords loading**:
   We load a list of Arabic stopwords to filter out commonly used but contextually insignificant words. This is an important step for improving the performance of the model, as stopwords do not contribute valuable information.

4. **Text cleaning**:
   We apply a series of text cleaning steps to standardize and simplify the text data. This involves:
   - **Removing links (URLs)**: Any URLs present in the text are removed as they are not meaningful for classification purposes.
   - **Removing special characters and punctuation**: This step removes any non-alphabetical characters, ensuring the text only contains meaningful words.
   - **Removing Arabic diacritics (Tashkeel) and elongated letters (Tatweel)**: Diacritical marks and elongated letters are stripped out to standardize the text.
   - **Removing Arabic stopwords**: Words that are part of the stopwords list are removed, as they do not add value to the classification task.
   - **Stemming or Lemmmatization**: Either stemming or lemmatization is applied to reduce words to their root or base form.
   - **Normalizing Hamza**: Any variation of the Hamza character is normalized for consistency.

   **Note:** Most of these you can do using the library [PyArabic](https://pyarabic.readthedocs.io/ar/latest/README.html#features)

5. **Final cleanup**:
   Apply the cleanup function to the feature column.

By following these steps, the text will be cleaned, filtered, and ready for tokenization!

In [ ]:
df.dropna()

,text,label
0,https://example.com/resource/الشاٌرقة -ْ محمِد...,Culture
1,https://example.com/resource/اَنِطٌلقّتَ ٍفٍيّ...,Culture
2,https://example.com/resource/أُقيًمٌتِ مِساءُ ...,Culture
3,https://example.com/resource/بٍاسُمةَ يًوٌنٍس ...,Culture
4,https://example.com/resource/قُرر اَتحِاد اًلْ...,Culture
...,...,...
39880,https://example.com/resource/أعلّنت شّركٌةً بّ...,Tech
39881,https://example.com/resource/بُتٍاَرٌيًخَ 28ْ ...,Tech
39882,https://example.com/resource/دبَيُ:َ «ُاَلخليج...,Tech
39883,https://example.com/resource/LٌG GًS2ْ9ً0 Coْo...,Tech


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords

arabic_stopwords = set(stopwords.words('arabic'))


In [ ]:
def filter_by_word_count(text, min_words=5, max_words=100):
    words = text.split()
    if min_words <= len(words) <= max_words:
        return text
    else:
        return None


In [ ]:
import re

def remove_links(text):
    return re.sub(r'http\S+|www.\S+', '', text)


In [ ]:
def remove_special_characters(text):
    return re.sub(r'[^أ-ي\s]', '', text)

In [ ]:
import pyarabic.araby as araby

def remove_tashkeel_and_tatweel(text):
    text = araby.strip_tashkeel(text)
    return araby.strip_tatweel(text)


In [ ]:
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in arabic_stopwords]
    return ' '.join(filtered_words)


In [ ]:
from nltk.stem.isri import ISRIStemmer

def stem_text(text):
    stemmer = ISRIStemmer()
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)


In [ ]:
def normalize_hamza(text):
    return araby.normalize_hamza(text)


In [ ]:
def clean_text(text):
    text = remove_links(text)
    text = remove_special_characters(text)
    text = remove_tashkeel_and_tatweel(text)
    text = normalize_hamza(text)
    text = remove_stopwords(text)
    text = stem_text(text)
    return text


# Tokenization, Padding, and Data Splitting

In this step, we will prepare the text data for input into a model by converting the text into numerical sequences, padding them to a uniform length, and splitting the dataset into training and testing sets. Here's an overview of the steps involved:

1. **Tokenization**:
   We use a tokenizer to convert the cleaned text into numerical sequences. You can use `Tokenizer` tokenizer from `tensorflow.keras.preprocessing.text` package or any other tokenizer you like.

2. **Text to sequences**:
   After fitting the tokenizer on the cleaned text, we transform each text into a sequence of numbers, where each number corresponds to a token (word) in the text.

3. **Padding the sequences**:
   Since different texts may vary in length, we pad the sequences to ensure they all have the same length.

4. **Label encoding**:
   The labels (target values) also need to be converted into numerical form if they are not encoded.

5. **Train-test split**:
   The dataset is divided into training and testing sets. We allocate 80% of the data for training the model and reserve 20% for testing its performance.
   
   - The **training data** consists of the padded sequences used to train the model.
   - The **training labels** are the encoded labels corresponding to the training data.
   - The **testing data** is used to assess the model’s performance after training.
   - The **testing labels** are the encoded labels corresponding to the testing data.

6. **Data shape confirmation**:
   After splitting the data, we print the shape (dimensions) of both the training and testing sets to confirm that the data is properly divided and formatted.

By the end of this step, the text data will be transformed into padded numerical sequences, the labels will be encoded, and the data will be split into training and testing sets for model development and evaluation.

In [ ]:

df['cleaned_text'] = df['text'].apply(lambda x: clean_text(x) if filter_by_word_count(x) else None)


In [ ]:
df = df.dropna(subset=['cleaned_text'])


In [ ]:
df['cleaned_text'] = df['cleaned_text'].fillna('')




In [111]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['cleaned_text'])
sequences = tokenizer.texts_to_sequences(df['cleaned_text'])

#print(f'The converted words in our the text vector of integers: \n {sequences}')

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

# Building the Classifier

In this step, you will design and build a NLP Classifier model to classify text data. Below is a breakdown of the key components you'll implement, but it's up to you to decide how to configure them based on your understanding and experimentation:

1. **Model Type**:
   You will use a Sequential model, which allows you to stack layers in a linear sequence.

2. **Input Layer**:
   Define the shape of the input data. Consider the dimensions of your padded sequences and set the input shape accordingly.

3. **Embedding Layer**:
   The embedding layer will convert input tokens (integers) into dense vector representations. You will need to determine the size of the input dimension (based on your vocabulary) and the output dimension (embedding size).

4. **Bidirectional Simple RNN/LSTM Layers**:
   You can add one or more recurrent layers. Consider using Bidirectional layers to capture contextual information from both directions (forward and backward). You can chose SimpleRNN/GRU/LSTM to perform this step.

5. **Dense Layers**:
   Add one or more fully connected (Dense) layers to process the output from the RNN/GRU/LSTM layers.

6. **Output Layer**:
   The output layer should match the type of classification task you're working on. Consider using appropriate activation function with appropriate number of units.

7. **Model Summary**:
   After defining your model architecture, print a summary to review the number of layers, types of layers, and total parameters.

8. **Model Compilation**:
   Finally, compile the model by selecting an optimizer, a loss function, and metrics.

In [73]:
from tensorflow.keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Bidirectional

vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 128
num_classes = 7

model = Sequential()

model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))

model.add(Bidirectional(LSTM(64, return_sequences=False)))


model.add(Dense(64, activation='relu'))


model.add(Dense(num_classes, activation='softmax'))



In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_8 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

# Defining Batch Size, Creating Datasets, and Training the Model

In this step, you will define the batch size, create TensorFlow Datasets for both training and testing, and train the model. The key elements to consider are outlined below, and it is up to you to choose the specific configurations based on your preferences and experimentation:

1. **Batch Size**:
   Select a batch size for training and testing. The batch size determines how many samples will be processed together in one forward and backward pass during training.

2. **Creating Datasets**:
   Use TensorFlow’s `Dataset.from_tensor_slices()` to create datasets from the training and testing data.

3. **Batching the Datasets**:
   Batch the datasets by grouping the data into batches of the specified size.

4. **Training the Model**:
   Train the model by fitting it on the training dataset for a specified number of epochs. You will also need to provide the validation data to monitor the model’s performance on unseen data during training.

5. **Tracking Training History**:
   During training, the model’s performance metrics (such as loss and accuracy) will be tracked over the epochs, and the results will be stored in the `history` object.

In [98]:
train_data = df['text']
train_labels = df['label']
test_data = df['cleaned_text']
test_labels = df['label']

In [90]:
train_data.shape

(6198,)

In [91]:
train_labels.shape

(6198,)

In [92]:
X_train.shape

(3966,)

In [93]:
y_train.shape

(992,)

In [110]:
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder



vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 128
num_classes = 7
batch_size = 32


tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(train_data)


X_train_seq = tokenizer.texts_to_sequences(train_data)
X_test_seq = tokenizer.texts_to_sequences(test_data)

X_train_padded = pad_sequences(X_train_seq, padding='post')
X_test_padded = pad_sequences(X_test_seq, padding='post')

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(train_labels)
y_test_encoded = label_encoder.transform(test_labels)

X_train, X_val, y_train, y_val = train_test_split(X_train_padded, train_labels, test_size=0.2)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test_padded, test_labels)).batch(batch_size)


model = models.Sequential([
    layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    layers.LSTM(128, return_sequences=True),
    layers.GlobalMaxPool1D(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_dataset,
                    epochs=10,
                    validation_data=val_dataset)



Epoch 1/10


UnimplementedError: Graph execution error:

Detected at node compile_loss/sparse_categorical_crossentropy/Cast defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  Fi

# Model Evaluation

Once the model is trained, the next step is to evaluate its performance on the testing dataset.

1. **Evaluate the Model**:
   You will use the `evaluate()` method to assess the model’s performance on the test dataset.

2. **Testing Dataset**:
   Ensure that the testing dataset is properly prepared and batched, just like the training dataset.

4. **Loss Curve**:
   A loss curve plots the loss values for both the training and validation datasets over the epochs.

In [104]:
results = model.evaluate(X_test, y_test, batch_size=batch_size)

UnimplementedError: Graph execution error:

Detected at node compile_loss/sparse_categorical_crossentropy/Cast defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  Fi

# Model Inference

In this step, you will use the trained model to make predictions on new, unseen data (inference). Here’s an outline of the key points:

1. **Create Test Sample**:
   Create a string to test your modelm the goal here is to give the model Before making predictions, ensure that the new data is preprocessed in the same way as the training data. This includes tokenization, padding, and any other transformations you applied during the data preprocessing step. The data can be single text to see the result of the prediction.

2. **Model Prediction**:
   Use the `predict()` method to feed new samples into the trained model and obtain predictions. The model will output probabilities or predicted class labels based on the type of classification task (binary or multi-class).

3. **Interpreting Predictions**:
   The model will return probabilities for each class.

# Notebook Question:
- How did you handle text preprocessing? Why did you choose this approach?

- Why did you choose this model design?

- Why did you pick this number of layers or units for the model?

- Why did you select these evaluation methods?

- Does your model show signs of overfitting or underfitting? How do you know?

- What changes could you make to improve the model and fix overfitting or underfitting?

Answer Here:



    How did you handle text preprocessing? Why did you choose this approach?

    Why did you choose this model design?

    Why did you pick this number of layers or units for the model?

    Why did you select these evaluation methods?

    Does your model show signs of overfitting or underfitting? How do you know?

    What changes could you make to improve the model and fix overfitting or underfitting?


1. How did you handle text preprocessing? Why did you choose this approach?


    By using the Tokenizer class from Keras to convert the raw text into sequences of integers.
    So when we apply padding to the sequences to ensure that all input samples have the same length. LSTM  require fixed-length inputs. Padding ensures that all input samples have the same length, which prevents input shape mismatches.

    The labels were converted from strings to integers using LabelEncoder to make them compatible with the sparse_categorical_crossentropy loss function.

    Tokenization is important for NLP tasks to convert words into numbers, which the model can process.
  .

2. Why did you choose this model design?

Model Design:

    
        The first layer in the model, is the Embedding Layer which converts integer-encoded words into dense vectors of a fixed size (embedding dimension). This layer helps in learning relationships between words.
        after that (LSTM) layer is used to capture the sequential dependencies in the text. LSTMs are good for handling time-series or sequence data like text.
        the next layer we need to  reduce the dimensionality by GlobalMaxPool1D. It reduces the model complexity and helps to avoid overfitting.
        Finally, The dense layers that can classify the input data based on learned features from the previous layers.

Why this design?

    Text data is high-dimensional and sparse. The embedding layer transforms this sparse input into a more useful, dense, and low-dimensional representation. and Since text data is sequential, the LSTM layer helps the model remember long-term dependencies between words.
    Pooling reduces the dimensions, and captures the most relevant features.
    The dense layer combines learned features into a multi-class output.

3. Why did you pick this number of layers or units for the model?

    The embedding dimension of 128 is a common choice that balances model complexity and performance. Larger values could lead to overfitting, and smaller values might not capture enough information.

    for LSTM 128 is a is a good starting point for many text classification problems. It’s large enough to capture sequential patterns but not so large that it would make training difficult or lead to overfitting.

    Pooling helps reduce the model complexity without losing much information.
    
    the  Dense Layer with 64 units provides enough capacity without overcomplicating the model.
    
    with the Output Layer 7 units The number of units corresponds to the number of classes in the classification task.

4. Why did you select these evaluation methods?

    Sparse Categorical Crossentropy Loss is sutibale for multi-class classification where labels are integer-encoded. It calculates the difference between the predicted class probabilities and the true class labels. during the training, I used a validation dataset to monitor the model performance on unseen data. This helps detect overfitting early by showing whether the model performance on the training data transfers well to new data.

5. Does your model show signs of overfitting or underfitting? How do you know?

To determine if the model shows signs of overfitting or underfitting, I would examine the following:

You can check for signs of overfitting by comparing the training and validation loss and accuracy. If the training loss decreases significantly but the validation loss does not, or if validation accuracy stalls, these are signs of overfitting.

6. What changes could you make to improve the model and fix overfitting or underfitting?
To Fix Overfitting:

    Add Dropout can help by preventing the model  memorizing the training data. and prevent overfitting. if i had more time I would design my model perfectly to ensure accurate results.